In [ ]:
import json
from datetime import datetime

import pandas as pd
from kafka import KafkaConsumer

# Create a Kafka consumer
consumer = KafkaConsumer(
    "stock_prices",
    bootstrap_servers="localhost:9092",
    value_deserializer=lambda v: json.loads(v.decode("utf-8")),
    auto_offset_reset="earliest",  # start from earliest messages
    enable_auto_commit=True,
    group_id="stock_price_group",
)

# Storage list
records = []


print("Listening for new temp messages... (Ctrl+C to stop)")

try:
    for msg in consumer:
        data = msg.value
        data["timestamp"] = datetime.now().isoformat(timespec="seconds")
        print("Received:", data)

        records.append(data)
        # Periodically save to CSV every 20 messages
        if len(records) % 20 == 0:
            df = pd.DataFrame(records)
            df.to_csv("stock_price.csv", index=False)
            print(f"Saved {len(records)} records to stock_price.csv")

except KeyboardInterrupt:
    print("\nStopped by user.")
    if records:
        df = pd.DataFrame(records)
        print(f"Final save: {len(records)} total records.")


In [ ]:
print("Listening for new temp messages... (Ctrl+C to stop)")

try:
    for msg in consumer:
        data = msg.value
        data["timestamp"] = datetime.now().isoformat(timespec="seconds")
        print("Received:", data)

        records.append(data)
        # Periodically save to CSV every 20 messages
        if len(records) % 20 == 0:
            df = pd.DataFrame(records)
            df.to_csv("stock_price.csv", index=False)
            print(f"Saved {len(records)} records to temperature_data.csv")

except KeyboardInterrupt:
    print("\nStopped by user.")
    if records:
        df = pd.DataFrame(records)
        print(f"Final save: {len(records)} total records.")


In [ ]:
# visualise

import matplotlib.pyplot as plt
import pandas as pd

# Read the CSV file containing stock price data
df = pd.read_csv("stock_price.csv")

# List of tickers to plot
tickers = ["AAPL", "MSFT", "GOOGL", "TSLA"]

# Create a plot with the specified figure size
plt.figure(figsize=(16, 4))

# Loop through each ticker and plot the data
for ticker in tickers:
    df_plot = df[df["ticker"] == ticker]  # Select data for one stock
    plt.plot(pd.to_datetime(df_plot["timestamp"]), df_plot["price"], label=ticker)

# Add legend for the tickers
plt.legend(tickers)

# Show the plot
plt.show()
